# 01 — Environment Setup
This notebook installs dependencies and verifies GPU and 4-bit quantization readiness for the Titans-MIRAS hybrid memory demo.

In [ ]:
# Check Python version and system info
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

try:
    import torch
    print("PyTorch available:", True)
    print("Torch version:", torch.__version__)
except Exception as e:
    print("PyTorch import failed:", e)

In [ ]:
# Verify GPU availability and details
import torch
cuda_ok = torch.cuda.is_available()
print("CUDA available:", cuda_ok)
if cuda_ok:
    device = torch.device("cuda")
    idx = torch.cuda.current_device()
    print("GPU name:", torch.cuda.get_device_name(idx))
    cap = torch.cuda.get_device_capability(idx)
    print("Compute capability:", cap)
    total_mem = torch.cuda.get_device_properties(idx).total_memory
    print("Total GPU memory (GB):", round(total_mem/1024**3, 2))
    x = torch.randn(1024, 1024, device=device)
    y = torch.matmul(x, x)
    print("GPU matmul ok:", y.shape)
else:
    print("Running on CPU; 4-bit may be limited.")

In [ ]:
# Test 4-bit quantization support (bitsandbytes)
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch

warnings.filterwarnings("ignore")
try:
    model_id = "gpt2"  # small proxy for environment validation
    quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,
                                      bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    tok = AutoTokenizer.from_pretrained(model_id)
    tok.pad_token = tok.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quant_config,
        trust_remote_code=True,
    )
    print("Loaded model with 4-bit quantization:", model_id)
    import math
    inputs = tok("Hello Titans-MIRAS!", return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=8)
    print("Generation:", tok.decode(out[0]))
except Exception as e:
    print("4-bit test failed (this can be expected on CPU-only environments):", e)

# Next steps
If the checks succeeded, proceed to the memory architecture in Notebook 2.